In [1]:
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#from scipy.stats import shapiro, kstest, poisson, chisquare, ttest_ind, levene, bartlett, sem, ppf
import scipy.stats as stats
from scipy.stats import shapiro, levene
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
from src import soporte as sp

In [3]:
df_dia = pd.read_csv("files/productos_dia.csv", index_col = 0)
df_precios = pd.read_csv("files/precios_productos.csv", index_col = 0)

In [4]:
df_dia.shape

(11494, 5)

In [5]:
df_dia.head()

,url,supermarket,name,description,product_id
2362486,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,MISTOL lavavajillas mano original botella 900 ml,NaN,9e2fda45e63337294f2a604178823026
2362487,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,DIA lavavajillas máquina premium todo en uno e...,NaN,244994c762813d38881cb118742152a2
2362488,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,FAIRY lavavajillas máquina platinum azul en cá...,NaN,8a5398b602e44b93a0c0a4a7b9d22b84
2362489,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,FAIRY lavavajillas máquina platinum plus limón...,NaN,2cd7ea87b1963adb51d8e1295f6c14d3
2362501,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,FAIRY lavavajillas mano concentrado aloe vera ...,NaN,4abac3d30922f81012d7ae0c2cd955f2


In [6]:
df_dia.duplicated(subset="product_id").sum()

np.int64(1998)

In [7]:
df_precios.shape

(1000000, 5)

In [8]:
df_precios[df_precios.duplicated(subset="product_id", keep=False)].sort_values(by="product_id")

,price,reference_price,product_id,reference_unit,insert_date
6220095,"3,49","7,35",0008afbaae96e6003d71de151a2ec67b,kg,2022-04-26 00:00:00
5041956,"3,49","7,35",0008afbaae96e6003d71de151a2ec67b,kg,2022-02-04 00:00:00
3163372,"3,39","7,14",0008afbaae96e6003d71de151a2ec67b,kg,2021-09-28 00:00:00
5445272,"3,49","7,35",0008afbaae96e6003d71de151a2ec67b,kg,2022-03-03 00:00:00
6943538,"3,49","7,35",0008afbaae96e6003d71de151a2ec67b,kg,2022-06-15 00:00:00
...,...,...,...,...,...
5293128,"4,95","0,33",fff3bcc496c06619a23befd1cd9003c9,ud,2022-02-21 00:00:00
7325546,"5,25","0,35",fff3bcc496c06619a23befd1cd9003c9,ud,2022-07-13 00:00:00
4268815,"4,95","0,33",fff3bcc496c06619a23befd1cd9003c9,ud,2021-12-13 00:00:00
2814416,"4,95","0,33",fff3bcc496c06619a23befd1cd9003c9,ud,2021-09-03 00:00:00


In [9]:
df_final2 = df_precios.merge(df_dia, on =["product_id"])
df_final2.sample(5)

,price,reference_price,product_id,reference_unit,insert_date,url,supermarket,name,description
316716,"1,35","45,0",5bfb7f83d904198604a60ed822b8a61b,kg,2022-08-06 00:00:00,https://www.dia.es/compra-online/despensa/sal-...,dia-es,DANI clavo en grano frasco 30 gr,NaN
721521,"5,79","7,72",74445ae08f5b23bb5b503b703513a5d9,l,2021-09-14 00:00:00,https://www.dia.es/compra-online/bodega/vino/t...,dia-es,ALTOS DE TAMARON vino tinto crianza DO Ribera ...,NaN
270720,"3,39","4,52",a1d3f468b1f4c975e0d7e96170a01ddf,l,2022-02-21 00:00:00,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,MAYORDOMO limpia mopas spray 750 ml,NaN
782817,"3,69","3,08",ac1de38711594ced8918287e5edc9a5a,kg,2022-08-10 00:00:00,https://www.dia.es/compra-online/mascotas/gato...,dia-es,AS bocaditos en gelatina para gatos adultos 12...,NaN
954563,"0,59","2,36",6ae43a4bf7eb2b3d980614c911fae24c,kg,2021-12-18 00:00:00,https://www.dia.es/compra-online/al-dia/verdur...,dia-es,DIA ensalada 4 estaciones bolsa 250 gr,NaN


In [10]:
df_final2.shape

(1264512, 9)

In [11]:
columnas = ["price", "reference_price"]

for i in columnas: 
    df_final2[i] = df_final2[i].str.replace(",", ".")

In [12]:
for i in columnas: 
    df_final2[i] = df_final2[i].astype(float)

In [13]:
df_final2.dtypes

price              float64
reference_price    float64
product_id          object
reference_unit      object
insert_date         object
url                 object
supermarket         object
name                object
description        float64
dtype: object

In [14]:
df_muestra = df_final2.sample(2500)

In [15]:
sp.intervalo(df_muestra, "price")

El intervalo del 0.95 para la columna price es 3.22 -> 3.6


In [16]:
df_final2.sample(1500)["price"].mean()

np.float64(3.213986666666667)

In [17]:
df_final2["date"] = df_final2["insert_date"].str.split(" ").str[0]
df_final2.head()

,price,reference_price,product_id,reference_unit,insert_date,url,supermarket,name,description,date
0,0.70,0.70,9b1d806ddaed2a3b4c842c361e9171a8,ud,2022-06-08 00:00:00,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,DIA SUPER PACO guantes satinados talla S bolsa...,NaN,2022-06-08
1,3.19,10.63,baf038a3f001812899adb8beaf4e0ec7,kg,2022-02-10 00:00:00,https://www.dia.es/compra-online/despensa/desa...,dia-es,KELLOGGS cereales special k con frutas rojas c...,NaN,2022-02-10
2,3.99,15.96,ca2fc753e5ea44e8c95888983c1dafe1,l,2022-04-11 00:00:00,https://www.dia.es/compra-online/cuidado-perso...,dia-es,DIA IMAQE leche solar infantil spf 50+ tubo 25...,NaN,2022-04-11
3,3.39,5.22,48c959cc94d6d82d1151189b2abe9e83,l,2022-06-19 00:00:00,https://www.dia.es/compra-online/cuidado-perso...,dia-es,SANEX gel de ducha dermo protector piel normal...,NaN,2022-06-19
4,3.39,5.22,48c959cc94d6d82d1151189b2abe9e83,l,2022-06-19 00:00:00,https://www.dia.es/compra-online/cuidado-perso...,dia-es,SANEX gel de ducha biome protect piel normal b...,NaN,2022-06-19


In [18]:
df_final2["date"] = pd.to_datetime(df_final2["date"])

In [19]:
df_final2.head()

,price,reference_price,product_id,reference_unit,insert_date,url,supermarket,name,description,date
0,0.70,0.70,9b1d806ddaed2a3b4c842c361e9171a8,ud,2022-06-08 00:00:00,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,DIA SUPER PACO guantes satinados talla S bolsa...,NaN,2022-06-08
1,3.19,10.63,baf038a3f001812899adb8beaf4e0ec7,kg,2022-02-10 00:00:00,https://www.dia.es/compra-online/despensa/desa...,dia-es,KELLOGGS cereales special k con frutas rojas c...,NaN,2022-02-10
2,3.99,15.96,ca2fc753e5ea44e8c95888983c1dafe1,l,2022-04-11 00:00:00,https://www.dia.es/compra-online/cuidado-perso...,dia-es,DIA IMAQE leche solar infantil spf 50+ tubo 25...,NaN,2022-04-11
3,3.39,5.22,48c959cc94d6d82d1151189b2abe9e83,l,2022-06-19 00:00:00,https://www.dia.es/compra-online/cuidado-perso...,dia-es,SANEX gel de ducha dermo protector piel normal...,NaN,2022-06-19
4,3.39,5.22,48c959cc94d6d82d1151189b2abe9e83,l,2022-06-19 00:00:00,https://www.dia.es/compra-online/cuidado-perso...,dia-es,SANEX gel de ducha biome protect piel normal b...,NaN,2022-06-19


In [20]:
df_final2.dtypes

price                     float64
reference_price           float64
product_id                 object
reference_unit             object
insert_date                object
url                        object
supermarket                object
name                       object
description               float64
date               datetime64[ns]
dtype: object

In [21]:
df_final2["diferencia"] = df_final2["reference_price"] - df_final2["price"]

In [22]:
df_final2.head(2)

,price,reference_price,product_id,reference_unit,insert_date,url,supermarket,name,description,date,diferencia
0,0.70,0.70,9b1d806ddaed2a3b4c842c361e9171a8,ud,2022-06-08 00:00:00,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,DIA SUPER PACO guantes satinados talla S bolsa...,NaN,2022-06-08,0.00
1,3.19,10.63,baf038a3f001812899adb8beaf4e0ec7,kg,2022-02-10 00:00:00,https://www.dia.es/compra-online/despensa/desa...,dia-es,KELLOGGS cereales special k con frutas rojas c...,NaN,2022-02-10,7.44


In [23]:
def obtener_diferencia(precio_referencia, precio):

    return precio_referencia - precio

In [24]:
obtener_diferencia(10, 7)

3

In [ ]:
# df_final2["diferencia_2"] = df_final2.apply(lambda fila: obtener_diferencia(fila["reference_price"], fila["price"]), axis = 1)

In [26]:
df_final2.head(2)

,price,reference_price,product_id,reference_unit,insert_date,url,supermarket,name,description,date,diferencia
0,0.70,0.70,9b1d806ddaed2a3b4c842c361e9171a8,ud,2022-06-08 00:00:00,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,DIA SUPER PACO guantes satinados talla S bolsa...,NaN,2022-06-08,0.00
1,3.19,10.63,baf038a3f001812899adb8beaf4e0ec7,kg,2022-02-10 00:00:00,https://www.dia.es/compra-online/despensa/desa...,dia-es,KELLOGGS cereales special k con frutas rojas c...,NaN,2022-02-10,7.44


In [27]:
df_final2["url"][0]

'https://www.dia.es/compra-online/cuidado-del-hogar/utensilios-de-limpieza/p/270115'

In [28]:
df_final2["url"][127]

'https://www.dia.es/compra-online/despensa/desayunos-y-dulces/chocolates-y-bombones/p/148474'

In [29]:
df_final2["url"][42]

'https://www.dia.es/compra-online/platos-preparados/ensaladas/p/278588'

In [30]:
def sacar_info(x):
    if "al-dia" in x:
        return "frescos", x.split("/")[5]

    elif "bulto-mixto" in x or "peso" in x or "caducidad" in x or "clasificacion-de-categorias" in x:
        return x.split("/")[5], x.split("/")[6]
        
    else:
        return x.split("/")[4], x.split("/")[5]

In [31]:
sacar_info('https://www.dia.es/compra-online/platos-preparados/ensaladas/p/278588')

('platos-preparados', 'ensaladas')

In [32]:
sacar_info('https://www.dia.es/compra-online/platos-preparados/ensaladas/p/278588')[0]

'platos-preparados'

In [33]:
sacar_info('https://www.dia.es/compra-online/platos-preparados/ensaladas/p/278588')[1]

'ensaladas'

In [36]:
df_final2["categoria"] = df_final2["url"].apply(sacar_info)[0]

ValueError: Length of values (2) does not match length of index (1264512)

In [37]:
df_final2[["categoria2","subcategoria"]] = df_final2.apply(lambda x: sacar_info(x["url"]), axis=1, result_type="expand")

In [38]:
df_final2.head(2)

,price,reference_price,product_id,reference_unit,insert_date,url,supermarket,name,description,date,diferencia,categoria,categoria2,subcategoria
0,0.70,0.70,9b1d806ddaed2a3b4c842c361e9171a8,ud,2022-06-08 00:00:00,https://www.dia.es/compra-online/cuidado-del-h...,dia-es,DIA SUPER PACO guantes satinados talla S bolsa...,NaN,2022-06-08,0.00,"(cuidado-del-hogar, utensilios-de-limpieza)",cuidado-del-hogar,utensilios-de-limpieza
1,3.19,10.63,baf038a3f001812899adb8beaf4e0ec7,kg,2022-02-10 00:00:00,https://www.dia.es/compra-online/despensa/desa...,dia-es,KELLOGGS cereales special k con frutas rojas c...,NaN,2022-02-10,7.44,"(despensa, desayunos-y-dulces)",despensa,desayunos-y-dulces


In [39]:
df_final2["price"].value_counts()

price
1.99     43466
2.99     33913
1.89     23453
0.99     22479
1.00     21433
         ...  
14.89        2
19.59        1
16.00        1
12.35        1
2.84         1
Name: count, Length: 1026, dtype: int64

In [40]:
df_final2["price"].mean()

np.float64(3.4070820317751553)

In [41]:
df_final2["reference_unit"].value_counts()

reference_unit
kg        697263
l         401337
ud        150232
lavado     12811
m           1253
Name: count, dtype: int64